In [2]:
import xarray as xr
import matplotlib.pyplot as plt
from glob import glob
import numpy as np
from karin import plot_karin_old

In [62]:
# version = 'v0_2_0'
#version = 'v0_3_0'
#version = 'v1_0_0'
version = 'v2_0'

# region = 'browsebasin'
# area = [120, -16, 125, -11] # Kimberley
# satpass = '534' #'228' #'325' #'534' #'047' #'256' #'353' #256 047 534 325 228
# ncfiles = sorted(glob(f'/home/jupyter-mrayson/data/SWOT/l3_karin_nadir_v2_0/cycle_0*/*_{satpass}_*.nc'))

# region = 'pilbara'
# area = [112, -21.5, 118, -15] # Pilbara
# satpass = '340' #'409' #'381' #'312' # '103' '312' '034'
#ncfiles = sorted(glob(f'/home/jupyter-mrayson/data/SWOT/l3_karin_nadir_v2_0/cycle_0*/*_{satpass}_*.nc'))

region = 'lombok'
area = [114, -12, 117.5, -8.8] 
satpass = 437 #437 131 006 409 034 312
ncfiles = sorted(glob(f'/home/jupyter-mrayson/data/SWOT/l3_karin_nadir_v2_0/cycle_0*/*_{satpass}_*.nc'))

# region = 'brunei'
# area = [109, 3, 113, 9] 
# ncfiles = sorted(glob('/home/jupyter-mrayson/data/SWOT/l3_karin_nadir/*_008*.nc'))

# region = 'southchinasea'
# area = [108, 9, 113, 18] 
# ncfiles = sorted(glob('/home/jupyter-mrayson/data/SWOT/l3_karin_nadir/*_008*.nc'))

# region = 'indoseas_pass021'
# area = [108, -11, 135, 9] 
# ncfiles = sorted(glob('/home/jupyter-mrayson/data/SWOT/l3_karin_nadir_v0_3/*_021*.nc'))

# region = 'ningaloo_pass062'
# area = [113, -23, 115, -20.5] # Ningaloo
# ncfiles = sorted(glob('/home/jupyter-mrayson/data/SWOT/l3_karin_nadir_v1_0/cycle_0*/*_062*.nc'))

# region = 'ningaloo_pass131'
# area = [113, -23, 115, -20.5] # Ningaloo
# ncfiles = sorted(glob('/home/jupyter-mrayson/data/SWOT/l3_karin_nadir_v1_0/cycle_0*/*_131*.nc'))
ncfiles

['/home/jupyter-mrayson/data/SWOT/l3_karin_nadir_v2_0/cycle_002/SWOT_L3_LR_SSH_Expert_002_006_20230811T063607_20230811T072733_v2.0.0.nc',
 '/home/jupyter-mrayson/data/SWOT/l3_karin_nadir_v2_0/cycle_003/SWOT_L3_LR_SSH_Expert_003_006_20230901T032112_20230901T041238_v2.0.0.nc',
 '/home/jupyter-mrayson/data/SWOT/l3_karin_nadir_v2_0/cycle_005/SWOT_L3_LR_SSH_Expert_005_006_20231012T205119_20231012T214246_v2.0.0.nc',
 '/home/jupyter-mrayson/data/SWOT/l3_karin_nadir_v2_0/cycle_006/SWOT_L3_LR_SSH_Expert_006_006_20231102T173625_20231102T182751_v2.0.0.nc',
 '/home/jupyter-mrayson/data/SWOT/l3_karin_nadir_v2_0/cycle_006/SWOT_L3_LR_SSH_Expert_006_009_20231102T201045_20231102T210212_v2.0.0.nc',
 '/home/jupyter-mrayson/data/SWOT/l3_karin_nadir_v2_0/cycle_006/SWOT_L3_LR_SSH_Expert_006_010_20231102T210212_20231102T215338_v2.0.0.nc',
 '/home/jupyter-mrayson/data/SWOT/l3_karin_nadir_v2_0/cycle_006/SWOT_L3_LR_SSH_Expert_006_015_20231103T011926_20231103T021052_v2.0.0.nc',
 '/home/jupyter-mrayson/data/SWOT/

In [63]:
ds = xr.open_dataset(ncfiles[0])
ds

<xarray.Dataset>
Dimensions:               (num_lines: 9860, num_pixels: 69, num_nadir: 1135)
Coordinates:
    latitude              (num_lines, num_pixels) float64 ...
    longitude             (num_lines, num_pixels) float64 ...
Dimensions without coordinates: num_lines, num_pixels, num_nadir
Data variables: (12/21)
    time                  (num_lines) datetime64[ns] ...
    calibration           (num_lines, num_pixels) float64 ...
    cross_track_distance  (num_pixels) float64 ...
    dac                   (num_lines, num_pixels) float32 ...
    internal_tide         (num_lines, num_pixels) float64 ...
    mdt                   (num_lines, num_pixels) float64 ...
    ...                    ...
    ugosa_unfiltered      (num_lines, num_pixels) float64 ...
    vgos                  (num_lines, num_pixels) float64 ...
    vgosa_filtered        (num_lines, num_pixels) float64 ...
    vgosa_unfiltered      (num_lines, num_pixels) float64 ...
    i_num_line            (num_nadir) int16 ...
    i_num_pixel           (num_nadir) int8 ...
Attributes: (12/42)
    Conventions:                     CF-1.7
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    cdm_data_type:                   Swath
    comment:                         Sea Surface Height measured by Altimetry
    data_used:                       SWOT KaRIn L2_LR_SSH PGC0/PIC0/PIC2 (NAS...
    doi:                             https://doi.org/10.24400/527896/A01-2023...
    ...                              ...
    geospatial_lon_min:              31.916308
    geospatial_lon_max:              198.845758
    date_modified:                   2025-01-07T12:07:20Z
    history:                         2025-01-07T12:07:20Z: Created by DUACS K...
    date_created:                    2025-01-07T12:07:20Z
    date_issued:                     2025-01-07T12:07:20Z

In [64]:
# Merge all dataset

from datetime import datetime
def add_time_dim(xda):
    #xda = xda.expand_dims(satpass = [datetime.now()])
    xda = xda.expand_dims(satpass = [xda.time.mean().values])
    return xda

drop_vars=['i_num_line','i_num_pixel']
dsall = xr.open_mfdataset(ncfiles, preprocess=add_time_dim, 
                          drop_variables=drop_vars,
                         )
dsall

OSError: [Errno -101] NetCDF: HDF error: '/home/jupyter-mrayson/data/SWOT/l3_karin_nadir_v2_0/cycle_006/SWOT_L3_LR_SSH_Expert_006_463_20231119T012743_20231119T021909_v2.0.0.nc'

In [ ]:
# Subsample
ds = dsall

selection = (
    (ds.longitude > area[0]) &
    (ds.longitude < area[2]) &
    (ds.latitude > area[1]) &
    (ds.latitude < area[3]))
selection = selection.compute()
ds_area = ds.where(selection, drop=True)
ds_area

In [ ]:
%%time
#ds_area.to_netcdf('/home/jupyter-mrayson/data/SWOT/Karin_l3_{}_complete_calval_{}.nc'.format(version, region))
outfile='/home/jupyter-mrayson/data/SWOT/Karin_l3_{}_interim_science_{}_{}.nc'.format(version, satpass, region)
ds_area.to_netcdf(outfile)
outfile

In [ ]:
plot_karin_old(ds_area,9, var1='ssha_unedited',var2='ssha_unfiltered')

In [ ]:
for ii in range(len(ncfiles)):
    outfile = '../FIGURES/SWOT/SWOT_Karin_l3_{}_{}_{}.png'.\
        format(satpass, region, ds_area.satpass.values[ii].astype(str)[0:10])
    fig,_,_ =plot_karin_old(ds_area,ii, var1='ssha_unedited',var2='ssha_unfiltered', outfile=outfile)
    
    print(outfile)
    del fig

In [46]:
ds_area

<xarray.Dataset>
Dimensions:               (satpass: 28, num_lines: 186, num_pixels: 69)
Coordinates:
  * satpass               (satpass) datetime64[ns] 2023-08-04T19:49:56.251565...
    latitude              (num_lines, num_pixels) float64 -11.99 ... -8.867
    longitude             (num_lines, num_pixels) float64 116.1 116.1 ... 117.8
Dimensions without coordinates: num_lines, num_pixels
Data variables: (12/19)
    time                  (satpass, num_lines, num_pixels) datetime64[ns] dask.array<chunksize=(1, 186, 69), meta=np.ndarray>
    calibration           (satpass, num_lines, num_pixels) float64 dask.array<chunksize=(1, 186, 69), meta=np.ndarray>
    cross_track_distance  (satpass, num_pixels, num_lines) float64 dask.array<chunksize=(1, 69, 186), meta=np.ndarray>
    dac                   (satpass, num_lines, num_pixels) float32 dask.array<chunksize=(1, 186, 69), meta=np.ndarray>
    internal_tide         (satpass, num_lines, num_pixels) float64 dask.array<chunksize=(1, 186, 69), meta=np.ndarray>
    mdt                   (satpass, num_lines, num_pixels) float64 dask.array<chunksize=(1, 186, 69), meta=np.ndarray>
    ...                    ...
    ugos                  (satpass, num_lines, num_pixels) float64 dask.array<chunksize=(1, 186, 69), meta=np.ndarray>
    ugosa_filtered        (satpass, num_lines, num_pixels) float64 dask.array<chunksize=(1, 186, 69), meta=np.ndarray>
    ugosa_unfiltered      (satpass, num_lines, num_pixels) float64 dask.array<chunksize=(1, 186, 69), meta=np.ndarray>
    vgos                  (satpass, num_lines, num_pixels) float64 dask.array<chunksize=(1, 186, 69), meta=np.ndarray>
    vgosa_filtered        (satpass, num_lines, num_pixels) float64 dask.array<chunksize=(1, 186, 69), meta=np.ndarray>
    vgosa_unfiltered      (satpass, num_lines, num_pixels) float64 dask.array<chunksize=(1, 186, 69), meta=np.ndarray>
Attributes: (12/42)
    Conventions:                     CF-1.7
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    cdm_data_type:                   Swath
    comment:                         Sea Surface Height measured by Altimetry
    data_used:                       SWOT KaRIn L2_LR_SSH PGC0/PIC0/PIC2 (NAS...
    doi:                             https://doi.org/10.24400/527896/A01-2023...
    ...                              ...
    geospatial_lon_min:              34.982333
    geospatial_lon_max:              201.913324
    date_modified:                   2025-01-07T15:21:50Z
    history:                         2025-01-07T15:21:50Z: Created by DUACS K...
    date_created:                    2025-01-07T15:21:50Z
    date_issued:                     2025-01-07T15:21:50Z

In [47]:
dt = ds_area.satpass[-1].values -ds_area.satpass[-2].values 
dt_days = dt.astype(float)*1e-9/86400
dt_days

20.864636664541635

In [27]:
fac=1
sig_m2 = fac/(12.42/24)
sig_s2 = fac/(12./24)

np.abs(sig_m2-sig_s2)

0.06763285024154575

In [21]:
1/np.abs(sig_m2-sig_s2)

14.785714285714317

In [22]:
1/(12.0/24-0.5*23.83/24)

282.3529411764663

In [23]:
f=12.42/24
fnyq = 0.5*(23.83/24)
f, fnyq

(0.5175, 0.4964583333333333)

In [88]:
f-fnyq

0.02104166666666668